In [1]:
import cv2
import mediapipe as mp
import os
import tensorflow as tf
import tensorflow.keras as keras
import modules.mod_faceDetection as fd

In [2]:
# Initiate facedetection model
face_detector = fd.FaceDetection(model_selection = 0, threshold = 0.7)

In [10]:
def get_face(bbox_dict, frame):
    x_min, y_min, w, h = bbox_dict.values()
    frame_h, frame_w, _ = frame.shape
    # get centerpoint
    center_w = x_min + w/2
    center_h = y_min + h/2
    centerpoint = int(center_w * frame_w), int(center_h * frame_h)
    # square size is max of width/length based on mediapipe bounding box + 10% more
    face_size = max(w * frame_w, h * frame_h)
    face_size = int(face_size + face_size*0.1)


    # print(face_size) CAN BE DELTED

    # square based on max frame_w or frame_h JUST TO CHECK IT IS CENTERED
    # cv2.circle(frame, centerpoint, radius = 0, color = (255, 0, 255), thickness = 2)


    # get the 4 corners of face
    face_size_half = int(face_size/2)
    tl = (centerpoint[0] - face_size_half, centerpoint[1] - face_size_half)
    bl = (centerpoint[0] - face_size_half, centerpoint[1] + face_size_half)
    tr = (centerpoint[0] + face_size_half, centerpoint[1] - face_size_half)
    br = (centerpoint[0] + face_size_half, centerpoint[1] + face_size_half)
    # Note that colors below are messed up because opencv uses BGR
    cv2.circle(frame, tl, radius = 2, color = (0,0,255), thickness = 2)
    cv2.circle(frame, bl, radius = 2, color = (0,255,0), thickness = 2)
    cv2.circle(frame, tr, radius = 2, color = (255,0,0), thickness = 2)
    cv2.circle(frame, br, radius = 2, color = (0,255,255), thickness = 2)

    # CROP

FOR TESTING

In [13]:
test = cv2.imread('test.jpg')
test_bbox = face_detector.get_bboxs(test)[0]
face_detector.draw_bbox(test_bbox[0], test_bbox[1], test)
get_face(test_bbox[1], test)
# test_reshape = cv2.resize(test, (200,200))
# test.shape
cv2.imshow('hi',test)
cv2.waitKey(0)
cv2.destroyAllWindows()

"ACTUAL" THING

In [9]:
# MAIN
# model paths
GENDER_MODEL_PATH = '../face/gender_model.hdf5'
# get models

gender_model = keras.models.load_model(GENDER_MODEL_PATH)
# param is 0, first webcam in list of webcams
cam = cv2.VideoCapture(0)

while cam.isOpened():
    # take frame from webcam
    success, frame = cam.read()
    # flip frame for selfie mode
    frame = cv2.flip(frame, 1)
    # check if loaded frame
    if not success:
        print("Image is donezo gonezo")
        continue

    # frame setup
    frame.flags.writeable = False # not writeable, pass by reference
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # rgb for mediapipe use
    # detect boxes
    bboxs = face_detector.get_bboxs(frame)

    # draw bbox on frame
    frame.flags.writeable = True # now need to draw on
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) # back to bgr for opencv

    if bboxs:
        for bbox in bboxs:
            #Draw box on face
            face_detector.draw_bbox(bbox[0], bbox[1], frame)

    # display frame
    cv2.imshow('Webcam', frame)

    # use q to quit
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# close the camera
cam.release()
# Close windows
cv2.destroyAllWindows()